# **Create Freesound Corpora**



# **Setup Collab**

In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    #pip install git+git://github.com/mtg/freesound-python.git#egg=freesound
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)

# **Imports**

In [2]:
import os
import pandas as pd
import numpy as np
import freesound
from IPython.display import display
import re
import time
from os import listdir

# **Setup project**

In [3]:

ROOT_PATH = "./"
if IN_COLAB:
    ROOT_PATH = "/content/drive/My Drive/Colab Notebooks/MIR"

FILES_DIR = os.path.join(ROOT_PATH, 'audio_corpora_freesound')
FILES_DIR_ANNOTATIONS = os.path.join(ROOT_PATH, 'annotations_corpora_freesound') 
DATAFRAME_FILENAME = os.path.join(ROOT_PATH, 'corpora_dataframe.csv')  


#FREESOUND
FREESOUND_API_KEY = 'z3BjEL4rNKquyDVOMCO8AIUx9HTQ74j0IfUvSLNF' 
FREESOUND_STORE_METADATA_FIELDS = ['id', 'name', 'username', 'previews', 'license', 'description']  
freesound_client = freesound.FreesoundClient()
freesound_client.set_token(FREESOUND_API_KEY)


#PROJECT OPTION 
MAX_ITEMS_FOR_BPM_CLASS = 100
MIN_BPM_CLASS = 40
MAX_BPM_CLASS = 240
MAX_LENGHT_IN_S = 180
DOWNLOAD = True

# **Define some util functions**

In [4]:
# Define some util functions

def get_ogg_filename(sound):
    return sound.previews.preview_hq_ogg.split("/")[-1]

def query_freesound(query, filter, num_results=10):
    """Queries freesound with the given query and filter values.
    If no filter is given, a default filter is added to only get sounds shorter than 30 seconds.
    """
    if filter is None:
        filter = 'duration:[0 TO 30]'  # Set default filter
    pager = freesound_client.text_search(
        query = query,
        filter = filter,
        fields = ','.join(FREESOUND_STORE_METADATA_FIELDS),
        group_by_pack = 1,
        page_size = num_results
    )
    return [sound for sound in pager]

def retrieve_sound_preview(sound, directory):
    """Download the high-quality OGG sound preview of a given Freesound sound object to the given directory.
    """
    return freesound.FSRequest.retrieve(
        sound.previews.preview_hq_ogg,
        freesound_client,
        os.path.join(directory, get_ogg_filename(sound))
    )

def make_pandas_record(sound, bpm):
    """Create a dictionary with the metadata that we want to store for each sound.
    """
    record = {key: sound.as_dict()[key] for key in FREESOUND_STORE_METADATA_FIELDS}
    del record['previews']  # Don't store previews dict in record
    record['file_id'] = record['id']  # Rename 'id' to 'file_id'
    del record['id']
    record['path'] =  os.path.join(FILES_DIR, get_ogg_filename(sound)) 
    record['bpm'] = bpm
    return record

def prepare_query_for_bpm(bpm):
    
    str_filter_1 = "tag:{} duration:[10 TO {}]".format(bpm,MAX_LENGHT_IN_S)
    str_filter_2 = "duration:[10 TO {}]".format(MAX_LENGHT_IN_S)
    str_query = "{} bpm".format(bpm)
    freesound_queries = [
    {
        'query': 'bpm',
        'filter': str_filter_1,
        'num_results': MAX_ITEMS_FOR_BPM_CLASS,
    },
    {
        'query': str_query,
        'filter': str_filter_2,
        'num_results': MAX_ITEMS_FOR_BPM_CLASS,
    },
    ]
    
    return freesound_queries

def bpm_check(source, bpm):
    
    name = source.as_dict()['name']
    description = source.as_dict()['description']
    
    regex = "(?<!\d){}+\s?bpm".format(bpm)   
    pattern = re.compile(regex)
    matched = False
    
    if pattern.search(name) or pattern.search(description):
        matched = True
        
    return matched

def process_query(freesound_queries):
    
    sounds = sum([query_freesound(query['query'], query['filter'], query['num_results']) for query in freesound_queries],[])
    
    if DOWNLOAD :
        # Download the sounds and save them to FILES_DIR folder
        for count, sound in enumerate(sounds):
            if  get_ogg_filename(sound) not in file_already_in_folder:
                print('Downloading sound with id {0} [{1}/{2}]'.format(sound.id, count + 1, len(sounds)))
                retrieve_sound_preview(sound, FILES_DIR)

    # Make a Pandas DataFrame with the metadata of our sound collection and save it
    sounds_filtered = []
    for s in sounds:
        if bpm_check(s, bpm):
            sounds_filtered.append(s)
    
    return sounds_filtered

# **Check for files already downloaded**

In [11]:
file_already_in_folder = []
if not os.path.exists(FILES_DIR): 
    os.mkdir(FILES_DIR)
else:
    for f in listdir(FILES_DIR): file_already_in_folder.append(f)
        

# **Create Freesound Corpora**

In [12]:
panda_records = []

for bpm in range(MIN_BPM_CLASS,MAX_BPM_CLASS):
    
    print("Query bpm:{}".format(bpm))
    time.sleep(2)
    
    freesound_queries = prepare_query_for_bpm(bpm)
    sounds_filtered = process_query(freesound_queries)
    [panda_records.append(make_pandas_record(s,bpm)) for s in sounds_filtered]
    
   
df = pd.DataFrame(panda_records)
df.sort_values("file_id", inplace = True) 
df.drop_duplicates(subset ="file_id", keep = "first", inplace = True) 
df.reset_index(drop=True, inplace=True)
df.to_csv(DATAFRAME_FILENAME)
print('Saved DataFrame with {0} entries! {1}'.format(len(df), DATAFRAME_FILENAME))

# Show the contents of our DataFrame (the metadata of our source collection)
display(df)

Query bpm:40
Query bpm:41
Query bpm:42
Query bpm:43
Query bpm:44
Query bpm:45
Query bpm:46
Query bpm:47
Query bpm:48


Query bpm:49
Query bpm:50
Query bpm:51
Query bpm:52
Query bpm:53
Query bpm:54
Query bpm:55
Query bpm:56
Query bpm:57
Query bpm:58
Query bpm:59
Query bpm:60
Query bpm:61


Query bpm:62
Query bpm:63
Query bpm:64
Query bpm:65
Query bpm:66
Query bpm:67
Query bpm:68
Query bpm:69
Query bpm:70
Query bpm:71
Query bpm:72


Query bpm:73
Query bpm:74
Query bpm:75
Query bpm:76
Query bpm:77
Query bpm:78
Query bpm:79
Query bpm:80


Query bpm:81
Query bpm:82
Query bpm:83
Query bpm:84
Query bpm:85


...

Query bpm:139
Query bpm:140
Query bpm:141
Query bpm:142
Query bpm:143
Query bpm:144
Query bpm:145


Query bpm:146
Query bpm:147
Query bpm:148
Query bpm:149
Query bpm:150
Query bpm:151
Query bpm:152
Query bpm:153
Query bpm:154
Query bpm:155
Query bpm:156
Query bpm:157
Query bpm:158
Query bpm:159
Query bpm:160


Query bpm:161
Query bpm:162
Query bpm:163
Query bpm:164
Query bpm:165
Query bpm:166
Query bpm:167
Query bpm:168
Query bpm:169
Query bpm:170
Query bpm:171
Query bpm:172
Query bpm:173
Query bpm:174
Query bpm:175
Query bpm:176
Query bpm:177
Query bpm:178
Query bpm:179
Query bpm:180


Query bpm:181
Query bpm:182
Query bpm:183
Query bpm:184
Query bpm:185
Query bpm:186
Query bpm:187
Query bpm:188
Query bpm:189
Query bpm:190
Query bpm:191
Query bpm:192
Query bpm:193
Query bpm:194
Query bpm:195
Query bpm:196
Query bpm:197
Query bpm:198
Query bpm:199
Query bpm:200
Query bpm:201
Query bpm:202
Query bpm:203
Query bpm:204
Query bpm:205
Query bpm:206
Query bpm:207
Query bpm:208
Query bpm:209
Query bpm:210
Query bpm:211
Query bpm:212
Query bpm:213
Query bpm:214
Query bpm:215
Query bpm:216
Query bpm:217
Query bpm:218
Query bpm:219
Query bpm:220
Query bpm:221
Query bpm:222
Query bpm:223
Query bpm:224
Query bpm:225
Query bpm:226
Query bpm:227
Query bpm:228
Query bpm:229
Query bpm:230
Query bpm:231
Query bpm:232
Query bpm:233
Query bpm:234
Query bpm:235
Query bpm:236
Query bpm:237
Query bpm:238
Query bpm:239
Saved DataFrame with 1352 entries! ./corpora_dataframe.csv


,bpm,description,file_id,license,name,path,username
0,90,Tenor sax sample recorded at 44.1khz by myself...,670,http://creativecommons.org/licenses/by-nc/3.0/,too damn funky 16 beat Dm 90.wav,./audio_corpora_freesound/670_586-hq.ogg,simondsouza
1,123,make music this beats\nprocessed beat loop,872,http://creativecommons.org/licenses/by/3.0/,beats_123bpm.aif,./audio_corpora_freesound/872_872-hq.ogg,kotatsu
2,90,90 bpm noisy distorted guitar loop recorded wi...,1376,http://creativecommons.org/licenses/sampling+/...,90_bpm_parker.wav,./audio_corpora_freesound/1376_838-hq.ogg,sleep
3,132,132bpm loop made with reaktor,1886,http://creativecommons.org/licenses/by-nc/3.0/,machine1.aif,./audio_corpora_freesound/1886_176-hq.ogg,virotic
4,126,126bpm 4/4. 5+ measures with a fill. recorded ...,1979,http://creativecommons.org/licenses/by/3.0/,rbh 126bpm Tabla 06.wav,./audio_corpora_freesound/1979_1112-hq.ogg,RHumphries
5,133,This is loop 1 in a series of 16 variations. T...,2615,http://creativecommons.org/licenses/by/3.0/,133 bpm ATTACK LOOP 04 electrified analog kit ...,./audio_corpora_freesound/2615_2518-hq.ogg,Jovica
6,150,"Smokin'\r\nharmonica from Austin, Texas. These...",2631,http://creativecommons.org/licenses/by-nc/3.0/,Harp1.wav,./audio_corpora_freesound/2631_5150-hq.ogg,TexasMusicForge
7,83,"ambientloop, recorded with real rhodes!!",4048,http://creativecommons.org/licenses/by/3.0/,ambientloop83bpm.wav,./audio_corpora_freesound/4048_7740-hq.ogg,RealRhodesSounds
8,100,layered waves arpegiated @100bpm in c,8139,http://creativecommons.org/licenses/sampling+/...,layerarp100bpm in c.wav,./audio_corpora_freesound/8139_18307-hq.ogg,natmoon
9,100,layered waves arpegiated @100bpm in c,8140,http://creativecommons.org/licenses/sampling+/...,layerdarp 100bpm in c.wav,./audio_corpora_freesound/8140_18307-hq.ogg,natmoon


## Write annotations on files

In [13]:
import ntpath


file_already_in_folder = []
if not os.path.exists(FILES_DIR_ANNOTATIONS): 
    os.mkdir(FILES_DIR_ANNOTATIONS)

df_data = pd.read_csv(open(DATAFRAME_FILENAME), index_col=0)

for index, row in df_data.iterrows():
            bpm = row['bpm']
            file_id_path = row['path']
            name = ntpath.basename(file_id_path).split('.')[0]
            file_name = str(name) + ".bpm"
            full_path = os.path.join(FILES_DIR_ANNOTATIONS, file_name)
            
            f = open(full_path, "a")
            f.write(str(bpm))
            f.close()
            print("write file: {}".format(file_name))


write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id

write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id

write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id

write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id 510823
write file_id

## Clean files 

In [22]:
files_in_folder =  listdir(FILES_DIR)
files_in_folder = [os.path.join(FILES_DIR, name) for name in files_in_folder]

df_data = pd.read_csv(open(DATAFRAME_FILENAME), index_col=0)
files_in_csv = df_data['path'].tolist()


count = 0
for file in files_in_folder:
    if file not in files_in_csv:
        os.remove(file)
        count = count+1
print("file deleted: {}".format(count))

file deleted: 0
